Ejercicio: evaluación de un modelo de regresión logística

En el ejercicio anterior, ajustamos un modelo de regresión logística simple para predecir la posibilidad de una avalancha. Esta vez, crearemos el mismo modelo y analizaremos más a fondo cómo comprender mejor los errores que comete.

Visualización de datos
Recordemos nuestros datos. Recuerde que estamos planeando entrenar un modelo que pueda predecir avalanchas en función de la cantidad de capas débiles de nieve.

In [ ]:
import pandas
!pip install statsmodels
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/avalanche.csv
import graphing # custom graphing code. See our GitHub repo for details

#Import the data from the .csv file
dataset = pandas.read_csv('avalanche.csv', delimiter="\t")

#Let's have a look at the data and the relationship we're going to model
print(dataset.head())

graphing.box_and_whisker(dataset, label_x="avalanche", label_y="weak_layers")

Parece que las avalanchas están asociadas a tener capas de nieve más débiles. Dicho esto, algunos días se registraron muchas capas débiles, pero no se produjo ninguna avalancha. Esto significa que nuestro modelo tendrá dificultades para ser extremadamente preciso usando solo esta función. Sin embargo, continuemos y volvamos a esto en un ejercicio futuro.

Antes de comenzar, debemos dividir nuestro conjunto de datos en conjuntos de entrenamiento y de prueba. Entrenaremos en el conjunto de entrenamiento y probaremos (lo adivinaste) en el conjunto de prueba.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset in an 75/25 train/test ratio. 
train, test = train_test_split(dataset, test_size=0.25, random_state=10)

print("Train size:", train.shape[0])
print("Test size:", test.shape[0])

Ajuste de un modelo

Ajustemos un modelo de regresión logística simple utilizando log-loss como una función de costo. Esta es una forma muy estándar de ajustar un modelo de clasificación, tan estándar que, de hecho, no necesitamos especificarlo en absoluto.

In [ ]:
import statsmodels.formula.api as smf

# Perform logistic regression.
model = smf.logit("avalanche ~ weak_layers", train).fit()

print("Model trained")

Evaluación del modelo con información resumida
Si usamos statsmodels, podemos obtener un resumen detallado sobre el modelo simplemente llamando a summary()

In [ ]:
model.summary()

Este resumen proporciona información detallada.

En la fila inferior hay dos datos útiles. El coeficiente para las capas_débiles es positivo, lo que significa que a medida que aumentan las capas_débiles, también aumenta la probabilidad de una avalancha. La columna del valor P es inferior a 0,05, lo que significa que el modelo confía en que las capas débiles son un predictor útil de avalanchas.

Sin embargo, el resto de esta tabla es difícil de entender para los principiantes, por lo que no está claro qué tan bien está funcionando nuestro modelo. Probemos de otra manera.

Evaluación del modelo visualmente
A veces, pero no siempre, podemos evaluar visualmente un modo de regresión logística. Tracemos nuestro modelo contra los datos reales en el conjunto de datos de prueba.

In [ ]:
def predict(weak_layers):
    return model.predict(dict(weak_layers=weak_layers))

graphing.scatter_2D(test, label_x="weak_layers", label_y="avalanche", trendline=predict)

Es difícil ver la forma de S de la línea de tendencia, porque la cantidad de capas débiles de nieve y la probabilidad de una avalancha solo están débilmente relacionadas. Si alejamos, podemos obtener una vista ligeramente mejor:

In [ ]:
graphing.scatter_2D(test, label_x="weak_layers", label_y="avalanche", x_range=[-20,20], trendline=predict)

Al revisar el gráfico anterior, podemos ver que nuestro modelo predecirá una avalancha cuando el número de capas débiles de nieve sea mayor a 5. Podemos decir esto porque el valor de la línea es 0.5 en x=5 (recuerde que en el anterior unidad definimos un umbral clasificador, de modo que las probabilidades superiores a 0,5 se clasificarían como verdaderas).

Es difícil saber cómo se relaciona esto con los puntos: los puntos se superponen y, por lo tanto, es difícil ver cuántos puntos están en 0 o en 1. ¿De qué otra manera podemos evaluar el modelo?

Evaluar con función de costo
Evaluemos nuestro modelo con una función de costo de pérdida logarítmica.

In [ ]:
from sklearn.metrics import log_loss

# Make predictions from the test set
predictions = model.predict(test)

# Calculate log loss
print("Log loss", log_loss(test.avalanche, predictions))

0.66 - ¿Qué significa eso? Esto podría ser útil para comparar dos modelos diferentes, pero es difícil comprender exactamente lo que esto significa para el rendimiento en el mundo real.

Evaluar la precisión
En su lugar, evalúemos la precisión. La precisión se refiere a la proporción de predicciones que el modelo acertó, después de convertir las predicciones de probabilidades a avalancha o no avalancha.

Lo primero que debe hacer es convertir las probabilidades en predicciones duras:

In [ ]:
import numpy

# Print a few predictions before we convert them to categories
print(f"First three predictions (probabilities): {predictions.iloc[0]}, {predictions.iloc[1]}, {predictions.iloc[2]}")

# convert to absolute values
avalanche_predicted = predictions >= 0.5

# Print a few predictions converted into categories
print(f"First three predictions (categories): {avalanche_predicted.iloc[0]}, {avalanche_predicted.iloc[1]}, {avalanche_predicted.iloc[2]}")

Ahora podemos calcular la precisión:

In [ ]:
# Calculate what proportion were predicted correctly
guess_was_correct = test.avalanche == avalanche_predicted
accuracy = numpy.average(guess_was_correct)

# Print the accuracy
print("Accuracy for whole test dataset:", accuracy)

Parece que está prediciendo la respuesta correcta el 61% de las veces. Esta es información útil. Sin embargo, ¿qué tipo de errores está cometiendo? Echemos un vistazo a si se trata de adivinar una avalancha cuando no la hay (falsos positivos) o no adivinar 'avalancha' cuando en realidad ocurre una (falso negativo):

In [ ]:
# False Negative: calculate how often it guessed no avalanche when one actually occurred
false_negative = numpy.average(numpy.logical_not(guess_was_correct) & test.avalanche)

# False positive: calculate how often it guessed avalanche, when none actually happened
false_positive = numpy.average(numpy.logical_not(guess_was_correct) & numpy.logical_not(test.avalanche))


print(f"Wrongly predicted an avalanche {false_positive * 100}% of the time")
print(f"Failed to predict avalanches {false_negative * 100}% of the time")

¡Creo que podemos estar de acuerdo en que es mucho más comprensible que la función de costo o el gráfico!

Resumen

Hemos repasado diferentes formas de evaluar un modelo de regresión logística. Hemos visto que los resúmenes detallados pueden proporcionar información valiosa, pero esto puede ser difícil de digerir. Las métricas para este tipo de modelos tampoco son necesariamente lo suficientemente intuitivas o detalladas para comprender el modelo. Con un poco de trabajo adicional, podemos comparar las predicciones reales con los resultados reales y obtener una intuición de cómo podría funcionar el modelo en el mundo real.